In [35]:
from collections import defaultdict
import numpy as np
import os
from tqdm import tqdm
import matplotlib
matplotlib.use('Agg')  # 不显示图片，仅保存图片
import matplotlib.pyplot as plt

from mmengine import dump
from pycocotools.coco import COCO

## 同一对sub|obj，同时有多种rel的概率

In [55]:
def comput_correletion_of_rel_per_obj(coco_ins: COCO, coco_rel: COCO):
    ins_categories = coco_ins.dataset['categories']
    rel_categories = coco_rel.dataset['categories']
    ins_cat2label = {cat['id']: i for i, cat in enumerate(ins_categories)}
    rel_cat2label = {cat['id']: i for i, cat in enumerate(rel_categories)}
    # corr_mat 关于次对角线对称（鉴于矩阵存储方案，即右上角为原点）
    corr_mats = np.zeros((len(ins_cat2label), len(rel_cat2label), len(rel_cat2label)), dtype=int)

    for img_id in tqdm(coco_rel.getImgIds()):
        correlation = defaultdict(set)
        for ann in coco_rel.loadAnns(coco_rel.getAnnIds(img_id)):
            correlation[(ann['subject_id'], ann['object_id'])].add(rel_cat2label[ann['category_id']])
        correlation = dict(correlation)

        for (sub_id, obj_id), rels in correlation.items():
            obj = ins_cat2label[coco_ins.loadAnns(obj_id)[0]['category_id']]
            rels = np.array(list(rels))
            rel_x, rel_y = np.meshgrid(rels, rels)
            corr_mats[obj, rel_x, rel_y] += 1

    corr_infos = dict()
    for obj, corr_mat in enumerate(corr_mats):
        valid_rels = np.where(corr_mat.sum(0))[0]
        rel_x, rel_y = np.meshgrid(valid_rels, valid_rels)
        obj_info = coco_ins.dataset['categories'][obj]
        
        corr_infos[obj_info['id']] = {
            'obj_info': obj_info,
            'rels_info': [rel_categories[i] for i in valid_rels],
            'corr_mat': corr_mat[rel_x, rel_y]
        }

    return corr_infos

In [70]:
def plt_correletion_of_obj(corr_infos: dict, save_dir: str = None):
    if save_dir is not None and not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for corr_info in corr_infos.values():
        obj = corr_info['obj_info']
        title = f"correletion of {obj['name']}"
        rels = [info['name'] for info in corr_info['rels_info']]
        num_rels = len(rels)
        corr_mat = corr_info['corr_mat']
        hoi_freq = np.diag(corr_mat)
        corr_mat = corr_mat / hoi_freq[None]

        fig, ax = plt.subplots(figsize=(2+num_rels/2, 2+num_rels/2))
        ax.set_title(title+f"(id: {obj['id']})")
        ax.imshow(corr_mat)
        ax.set_xticks(range(num_rels))
        ax.set_yticks(range(num_rels))
        ax.set_xticklabels(rels)
        ax.set_yticklabels([f"{rel}-{freq}" for rel, freq in zip(rels, hoi_freq)])
        # 横坐标标签旋转45°
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
        # 在每个区域上打印对应数值
        for i in range(num_rels):
            for j in range(num_rels):
                data = corr_mat[i, j]
                if data < 0.5:
                    ax.text(j, i, f'{data:.2f}', ha="center", va="center", color="w")
                else:
                    ax.text(j, i, f'{data:.2f}', ha="center", va="center", color="black")
        fig.tight_layout()
        if save_dir is not None:
            fig.savefig(os.path.join(save_dir, title.replace(' ', '_')+'.jpg'), dpi=200)
        else:
            fig.show()

### raw anns

In [76]:
train_ins = COCO('../../annotations/instances_train.json')
train_rel = COCO('../../annotations/relations_train.json')
test_ins = COCO('../../annotations/instances_test.json')
test_rel = COCO('../../annotations/relations_test.json')

loading annotations into memory...
Done (t=0.97s)
creating index...
index created!
loading annotations into memory...
Done (t=0.14s)
creating index...
index created!
loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [77]:
corr_infos_train = comput_correletion_of_rel_per_obj(train_ins, train_rel)
corr_infos_test = comput_correletion_of_rel_per_obj(test_ins, test_rel)

100%|██████████| 9546/9546 [00:00<00:00, 17955.07it/s]


In [78]:
dump(corr_infos_train, 'correletion_train.pkl')
dump(corr_infos_test, 'correletion_test.pkl')

In [79]:
corr_infos_train[5]

{'obj_info': {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'},
 'rels_info': [{'id': 5, 'name': 'board'},
  {'id': 18, 'name': 'direct'},
  {'id': 26, 'name': 'exit'},
  {'id': 31, 'name': 'fly'},
  {'id': 42, 'name': 'inspect'},
  {'id': 53, 'name': 'load'},
  {'id': 58, 'name': 'no_interaction'},
  {'id': 77, 'name': 'ride'},
  {'id': 88, 'name': 'sit_on'},
  {'id': 112, 'name': 'wash'}],
 'corr_mat': array([[176,   0,   1,   0,   5,   5,   0,   0,   0,   0],
        [  0,  98,   0,   4,   1,   0,   0,   3,   6,   0],
        [  1,   0,  56,   1,   2,   1,   0,   2,   0,   0],
        [  0,   4,   1, 181,   2,   1,   0,  24,  34,   0],
        [  5,   1,   2,   2, 198,   5,   0,   0,   1,   0],
        [  5,   0,   1,   1,   5,  75,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0, 274,   0,   0,   0],
        [  0,   3,   2,  24,   0,   0,   0, 284,  49,   0],
        [  0,   6,   0,  34,   1,   0,   0,  49, 319,   0],
        [  0,   0,   0,   0,   0,   0,   0,  

In [ ]:
plt_correletion_of_obj(corr_infos_train, 'correletion_train')

In [ ]:
plt_correletion_of_obj(corr_infos_test, 'correletion_test')

### merged anns

In [ ]:
train_ins_merged = COCO('../../annotations/instances_train_merged.json')
train_rel_merged = COCO('../../annotations/relations_train_merged.json')
test_ins_merged = COCO('../../annotations/instances_test_merged.json')
test_rel_merged = COCO('../../annotations/relations_test_merged.json')

loading annotations into memory...
Done (t=0.51s)
creating index...
index created!
loading annotations into memory...
Done (t=0.16s)
creating index...
index created!
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [ ]:
corr_infos_train_merged = comput_correletion_of_rel_per_obj(train_ins_merged, train_rel_merged)
corr_infos_test_merged = comput_correletion_of_rel_per_obj(test_ins_merged, test_rel_merged)

100%|██████████| 9545/9545 [00:00<00:00, 18676.99it/s]


In [ ]:
dump(corr_infos_train_merged, 'correletion_train_merged.pkl')
dump(corr_infos_test_merged, 'correletion_test_merged.pkl')

In [ ]:
corr_infos_train_merged[5]

{'obj_info': {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'},
 'rels_info': [{'id': 5, 'name': 'board'},
  {'id': 18, 'name': 'direct'},
  {'id': 26, 'name': 'exit'},
  {'id': 31, 'name': 'fly'},
  {'id': 42, 'name': 'inspect'},
  {'id': 53, 'name': 'load'},
  {'id': 58, 'name': 'no_interaction'},
  {'id': 77, 'name': 'ride'},
  {'id': 88, 'name': 'sit_on'},
  {'id': 112, 'name': 'wash'}],
 'corr_mat': array([[176,   0,   1,   0,   7,   9,   0,   1,   0,   0],
        [  0,  98,   0,   5,   1,   0,   0,   3,   8,   0],
        [  1,   0,  56,   1,   2,   1,   0,   2,   0,   0],
        [  0,   5,   1, 181,   2,   1,   0,  43,  48,   0],
        [  7,   1,   2,   2, 198,  10,   0,   0,   2,   0],
        [  9,   0,   1,   1,  10,  75,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0, 274,   0,   0,   0],
        [  1,   3,   2,  43,   0,   0,   0, 284,  82,   0],
        [  0,   8,   0,  48,   2,   0,   0,  82, 319,   0],
        [  0,   0,   0,   0,   0,   0,   0,  

In [ ]:
plt_correletion_of_obj(corr_infos_train_merged, 'correletion_train_merged')

In [ ]:
plt_correletion_of_obj(corr_infos_test_merged, 'correletion_test_merged')

## TODO: 身体部位和HOI的相关性

In [8]:
parts_info = ['right_foot', 'right_leg', 'left_leg', 'left_foot', 'hip', 'head', 'right_hand', 'right_arm', 'left_arm', 'left_hand']

In [9]:
def comput_corr_of_parts_per_hoi(anns):
    correlation_of_parts_per_hoi = np.zeros((len(parts_info), len(hoi_info)), dtype=int)

    for item in anns:
        for hoi in item['hoi_annotations']:
            hoi_id = hoi_info[str(hoi['hoi_category_id'])]['valid_id']
            parts = list({pasta['part'] for pasta in hoi['part_state']})
            correlation_of_parts_per_hoi[parts, hoi_id] += 1

    return correlation_of_parts_per_hoi

In [140]:
def plt_correletion_of_part(correlation_of_parts_per_hoi, hoi_frequency, save_path=None):
    hoi_frequency = np.array([hoi_frequency[i] for i in range(1, 1+len(hoi_info))])
    correlation_of_parts_per_hoi = correlation_of_parts_per_hoi / (hoi_frequency[None, :] + 1e-5)

    xtick_labels = [f"{hoi['label']}-{hoi_frequency[hoi['valid_id']]}" for hoi in hoi_info.values()]
    ytick_labels = parts_info

    fig, ax = plt.subplots(figsize=(300, 8))
    im = ax.imshow(correlation_of_parts_per_hoi, vmin=0., vmax=1.)

    # We want to show all ticks...
    ax.set_xticks(np.arange(len(xtick_labels)))
    ax.set_yticks(np.arange(len(ytick_labels)))
    # ... and label them with the respective list entries
    ax.set_xticklabels(xtick_labels)
    ax.set_yticklabels(ytick_labels)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    for i in range(len(ytick_labels)):
        for j in range(len(xtick_labels)):
            data = correlation_of_parts_per_hoi[i, j]
            if data < 0.5:
                text = ax.text(j, i, f'{data:.2f}', ha="center", va="center", color="w")
            else:
                text = ax.text(j, i, f'{data:.2f}', ha="center", va="center", color="black")

    ax.set_title(f"Correletion of parts per hoi")
    fig.tight_layout()
    if save_path is not None:
        plt.savefig(save_path)
    else:
        plt.show()

In [10]:
train_pasta = json.load(open('../annotations/train_pasta.json'))
correlation_of_parts_per_hoi_train = comput_corr_of_parts_per_hoi(train_pasta)

# plt_correletion_of_part(correlation_of_parts_per_hoi_train, hoi_frequency_train, save_path='../correletion/parts_per_hoi_train')

### 身体部位和对象的距离与HOI的相关性

### 身体部位状态和HOI的相关性